In [18]:
import numpy as np
from scipy.linalg import expm
import numpy.linalg as LA
import random

from qiskit import QuantumCircuit
from qiskit import Aer, transpile,execute
from qiskit.quantum_info import random_pauli, state_fidelity, diamond_norm, Choi
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.tools.visualization import plot_histogram, plot_state_city

def create_hamiltonian(Nq, cn, gamma, Bn):
    '''
    create Hamiltonian gate
    target Hamiltonian is H=1/2*cn((1+gamma)XX+(1-gamma)YY)+BnZn
    Nq -> number of qubit
    cn -> coupling constant
    gamma -> parameter
    Bn -> Magnetic field
    '''
    
    XX= np.array(np.zeros(2**Nq))
    YY= np.array(np.zeros(2**Nq))
    Zn= np.array(np.zeros(2**Nq))
    Identity = 'I' * Nq
    hamiltonian = np.array(np.zeros(2**Nq))
    
    for k in range(0, Nq):
        '''
        隣あうqubitのみ相互作用するようなモデルを考える
        0 1 2 ... k-1 X X k+2 ... N-1
        0 1 2 ... k-1 Y Y k+2 ... N-1
        '''
        
        # 端まで計算したらスキップ
        if k == Nq-1:
            continue

        hamiX = Pauli(Identity[:k] + 'XX' + Identity[k+2:])
        hamiY = Pauli(Identity[:k] + 'YY' + Identity[k+2:])

        XX = XX + 0.5*cn[k]*(1+gamma)*hamiX.to_matrix()
        YY = YY + 0.5*cn[k]*(1-gamma)*hamiY.to_matrix()

    for k in range(0, Nq):
        hamiZ = Pauli(Identity[:k] + 'Z' + Identity[k+1:])
        Zn = Zn + Bn[k] * hamiZ.to_matrix()
    
    return XX + YY + Zn
        

In [19]:
def create_choi(clen, q, cn, r, bn, p, direct=True):
    '''
    loop -> loop count
    q -> number of qubit
    cn -> coupling constant
    r -> gamma parameter
    bn -> magnetic field
    p -> probability of random
    direct -> direct or indirect control
    '''
    
    qc = QuantumCircuit(q)
    
    for i in range(clen):
        t = 1
        # Hamiltonianの時間発展を計算
        hami = expm(-1j*create_hamiltonian(q,cn,r,bn)*t)
        qc.append(Operator(hami),list(range(q)))

        # randomな確率でPauliゲート追加
        # 各bitに独立な確率でPauliゲートを追加
        if random.randint(1, 100) < p:
            if direct:
                qc.append(random_pauli(1), [random.randint(0,q-1)])
            else:
                qc.append(random_pauli(1), [random.randint(0,1)])

    return Choi(qc).data

In [22]:
import datetime

def run():
    qubit = 5
    cn = [1] * qubit #[1,1,1,1,1]
    r = 0
    bn = [0] * qubit #[0,0,0,0,0]

    circuit_length = 500
    choi_qubit = 2 * qubit
    choi_direct = np.array(np.zeros(2**choi_qubit))
    choi_indirect = np.array(np.zeros(2**choi_qubit))

    loop = 10
    ## 時間をパラメータ化する
    ## totalの時間は揃えないとだめ(これを一定にしてランダムに時間をとる)
    for i in range(0, loop):
        print(datetime.datetime.now())
        choi_direct = choi_direct + create_choi(circuit_length, qubit, cn, r, bn, 3, True)
        choi_indirect = choi_direct + create_choi(circuit_length, qubit, cn, r, bn, 6, False)

#     print("choi_direct: %s" % choi_direct)
#     print("choi_indirect: %s" % choi_indirect)
    eps = (choi_direct - choi_indirect) / loop
    print(diamond_norm(Choi(eps)))
        

In [23]:
run()

2021-12-17 00:32:11.509751
2021-12-17 01:12:57.565452
2021-12-17 01:53:23.729801
2021-12-17 02:33:44.595085
2021-12-17 03:14:05.372937
2021-12-17 03:54:42.242067
2021-12-17 04:35:06.314510
2021-12-17 05:15:25.321431
2021-12-17 05:55:51.179040
2021-12-17 06:36:13.619205
0.0999999999992709


In [8]:
for i in range(0,1):
    print(i)

0
